In [4]:
import itertools
import os.path

## prepare arff files

In [2]:
ls = [25, 30]
methods = ["norsnet", "bval", "ucon"]

In [3]:
for l, m in itertools.product(ls, methods):
    path = "data/testset{}.{}".format(l, m)
    files = !find {path} -iname "*.fa"
    ids = []
    for file in files:
        filename
    with open("{}/testset{}.{}.arff".format(path, l, m), 'w') as arfffile:
        arfffile.write("@relation docs\n")
        arfffile.write("@attribute identifier {" + sIds + "}\n")
        arfffile.write("@attribute class {positive,negative}\n")
        arfffile.write("@data\n")
        for fid, group in ids: 
            arfffile.write(fid[1:] + "," + group + "\n")
            

data/testset25.norsnet
data/testset25.bval
data/testset25.ucon
data/testset30.norsnet
data/testset30.bval
data/testset30.ucon


## run weka

In [ ]:
l = 1
y = 4

In [ ]:
java -Xmx5g -cp /mnt/opt/data/pp1_14_exercise/wekaTobi.jar:/mnt/opt/data/pp1_14_exercise/weka.jar: weka.classifiers.meta.CVParameterSelection -t /path/to/train_arff.file -T /path/to/test_arff.file -S 1 -W weka.classifiers.functions.SMO -- -C 1.0 -L 0.0010 -P 1.0E-12 -N 0 -V -1 -W 1 -K "weka.classifiers.functions.supportVector.CustomPrecomputedKernelMatrixKernelFast -M /path/to/normalized_matrix.txt"


In [5]:
os.path.basename("/fo/baar/x.y.z")

'x.y.z'